In [ ]:
import os

def load_md_files(md_folder_path):
    """
    Lädt alle .md-Dateien aus einem angegebenen Ordner und gibt sie als Liste von Dokumenten zurück.

    :param md_folder_path: Der Pfad zum Ordner, der die .md-Dateien enthält.
    :return: Eine Liste mit dem Inhalt aller .md-Dateien.
    """
    documents = []

    # Durchlaufe alle Dateien im angegebenen Ordner
    for filename in os.listdir(md_folder_path):
        # Überprüfen, ob die Datei eine .md-Datei ist
        if filename.endswith('.md'):
            # Vollständigen Pfad zur Datei erstellen
            file_path = os.path.join(md_folder_path, filename)
            # Datei öffnen und Inhalt lesen
            with open(file_path, 'r', encoding='utf-8') as file:
                # Inhalt zur Dokumentenliste hinzufügen
                documents.append(file.read())

    return documents

# Beispiel: Die Funktion aufrufen
md_folder_path = os.path.join('data', 'md')
documents = load_md_files(md_folder_path)


for i, doc in enumerate(documents):
    print(f"Document {i}:\n")
    print(doc)
    print("\n" + "="*50 + "\n")

In [ ]:
import re
import language_tool_python

# Lade das LanguageTool-Modell für Deutsch
tool = language_tool_python.LanguageTool('de')

# Funktion zur Rechtschreib- und Grammatikprüfung mit LanguageTool
def correct_with_languagetool(text):
    matches = tool.check(text)
    corrected_text = language_tool_python.utils.correct(text, matches)
    return corrected_text

# Korrigieren von gesplitteten Wörtern
def correct_word_splitting(text):
    return re.sub(r'\b(\w+)\s*-\s*(\w+)\b', r'\1\2', text)

# Hauptfunktion zur Bereinigung der Dokumente
def clean_documents(documents):
    cleaned_documents = []
    
    for doc in documents:
        # Entfernen von problematischen Steuerzeichen (z.B. Steuerzeichen, nicht druckbare Zeichen)
        doc = re.sub(r'[\x00-\x1F\x7F]', ' ', doc)  # Entfernt Steuerzeichen
        
        # Sicherstellen, dass keine doppelten Leerzeichen durch das Entfernen von Zeichen entstehen
        doc = re.sub(r'\s+', ' ', doc).strip()
        
        # Korrigieren von gesplitteten Wörtern
        doc = correct_word_splitting(doc)
        
        # Textkorrektur mit LanguageTool
        doc = correct_with_languagetool(doc)
        
        # Bereinigtes Dokument zur Liste hinzufügen
        cleaned_documents.append(doc)
    
    return cleaned_documents

cleaned_documents = clean_documents(documents)

    
for i, doc in enumerate(cleaned_documents):
    print(f"cleaned_documents {i}:\n")
    print(doc)
    print("\n" + "="*50 + "\n")

In [ ]:
import re

def clean_text(documents):
    # Erlaubte Zeichen: Buchstaben (inkl. ä, ö, ü, ß), Ziffern, Bindestrich, Punkt, Komma, Ausrufezeichen und Fragezeichen
    pattern = r'[^a-zA-ZäöüÄÖÜß0-9\s.,!?-]'
    
    # Entfernt alle unerwünschten Zeichen aus jedem Dokument
    cleaned_documents = [re.sub(pattern, '', doc) for doc in documents]
    return cleaned_documents


# Bereinigte Dokumente
cleaned_documents_2 = clean_text(cleaned_documents)

# Ausgabe der bereinigten Dokumente
for doc in cleaned_documents_2:
    print(doc)

In [ ]:
import spacy

def chunk_documents(documents):
    """
    Segmentiert eine Liste von Texten in Sätze.
    
    :param documents: Liste von Texten (Strings), die segmentiert werden sollen
    :return: Liste von Listen, wobei jede innere Liste die Sätze eines Dokuments enthält
    """
    # Lade das deutsche Modell
    nlp = spacy.load('de_core_news_sm')
    
    # Liste zum Speichern der segmentierten Dokumente
    chunked_documents = []
    
    # Verarbeitung und Segmentierung jedes Dokuments in Sätze
    for text in documents:
        doc = nlp(text)
        sentences = list(doc.sents)
        chunked_documents.append(sentences)
    
    return chunked_documents

# Aufruf der Funktion
result = chunk_documents(cleaned_documents_2)

# Ausgabe der Resultate
for i, doc in enumerate(result):
    print(f"Dokument {i+1}:")
    for sentence in doc:
        print(sentence)
    print("\n")

In [ ]:
# Ausgabe der Chunks
for i, doc in enumerate(result):
    print(f"Chunks für Dokument {i+1}:")
    for j, sentence in enumerate(doc):
        print(f"  Chunk {j+1}: {sentence}")
    print("\n")

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def adjacent_sentence_clustering(documents, similarity_threshold=0.5):
    """
    Cluster adjacent sentences in the documents based on cosine similarity.

    Args:
    documents (list of str): List of documents (each document is a string of sentences).
    similarity_threshold (float): Threshold to determine if sentences should be in the same cluster.

    Returns:
    list of list of str: List of clusters (chunks) for each document.
    """
    clusters = []
    
    for doc in documents:
        sentences = doc.split('. ')
        if len(sentences) == 1:  # Falls das Dokument nur einen Satz enthält
            clusters.append([doc])
            continue
        
        vectorizer = TfidfVectorizer().fit_transform(sentences)
        vectors = vectorizer.toarray()
        
        clusters_in_doc = []
        current_cluster = [sentences[0]]
        
        for j in range(1, len(sentences)):
            similarity = cosine_similarity(vectors[j - 1].reshape(1, -1), vectors[j].reshape(1, -1))[0][0]
            
            if similarity >= similarity_threshold:
                current_cluster.append(sentences[j])
            else:
                clusters_in_doc.append(' '.join(current_cluster))
                current_cluster = [sentences[j]]
        
        # Add the last cluster
        if current_cluster:
            clusters_in_doc.append(' '.join(current_cluster))
        
        clusters.append(clusters_in_doc)
    
    return clusters


chunks = adjacent_sentence_clustering(cleaned_documents_2, similarity_threshold=0.5)
for i, chunk in enumerate(chunks):
    print(f"Dokument {i+1} hat die folgenden Chunks:")
    for c in chunk:
        print(f"- {c}")

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering, DBSCAN
import spacy
import numpy as np

# Lade das deutsche Sprachmodell von spaCy
nlp = spacy.load('de_core_news_sm')

def split_sentences(text):
    """
    Trennt den Text in Sätze unter Verwendung von spaCy.
    """
    doc = nlp(text)
    return [sent.text.strip() for sent in doc.sents]

def preprocess_text(text):
    """
    Vorverarbeitung des Textes, um unerwünschte Zeichen zu entfernen.
    """
    text = text.replace('\n', ' ').replace('  ', ' ').strip()
    return text

def semantic_clustering(documents, method='agglomerative', similarity_threshold=0.5, eps=0.5):
    """
    Führt eine semantische Clusterung von Sätzen in Dokumenten durch.
    
    Args:
    documents (list of str): Liste von Dokumenten, wobei jedes Dokument eine Zeichenkette ist.
    method (str): Clustering-Methode ('agglomerative' oder 'dbscan').
    similarity_threshold (float): Schwellenwert für die Ähnlichkeit (nur für Agglomerative Clustering).
    eps (float): Epsilon-Wert für DBSCAN (nur für DBSCAN).

    Returns:
    list of list of str: Liste von Clustern (Chunks) für jedes Dokument.
    """
    model = SentenceTransformer('all-MiniLM-L6-v2')
    clusters = []

    for doc in documents:
        doc = preprocess_text(doc)
        sentences = split_sentences(doc)
        
        if len(sentences) == 1:
            clusters.append([doc])
            continue

        embeddings = model.encode(sentences)

        if method == 'agglomerative':
            clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=1 - similarity_threshold)
        elif method == 'dbscan':
            clustering_model = DBSCAN(eps=eps, min_samples=2, metric='cosine')
        else:
            raise ValueError("Unsupported clustering method. Choose 'agglomerative' or 'dbscan'.")

        clustering_model.fit(embeddings)
        cluster_labels = clustering_model.labels_

        clustered_sentences = {}
        for sentence, label in zip(sentences, cluster_labels):
            if label == -1:  # DBSCAN Outlier
                continue
            if label not in clustered_sentences:
                clustered_sentences[label] = []
            clustered_sentences[label].append(sentence)
        
        clusters_in_doc = [' '.join(clustered_sentences[label]) for label in np.unique(cluster_labels) if label != -1]
        clusters.append(clusters_in_doc)

    return clusters


# Verwende Agglomerative Clustering
chunks_agglomerative = semantic_clustering(cleaned_documents_2, method='agglomerative', similarity_threshold=0.75)
for i, chunk in enumerate(chunks_agglomerative):
    print(f"Dokument {i+1} hat die folgenden Chunks (Agglomerative Clustering):")
    for c in chunk:
        print(f"- {c}")

# Verwende DBSCAN
chunks_dbscan = semantic_clustering(cleaned_documents_2, method='dbscan', eps=0.3)
for i, chunk in enumerate(chunks_dbscan):
    print(f"Dokument {i+1} hat die folgenden Chunks (DBSCAN):")
    for c in chunk:
        print(f"- {c}")

In [ ]:
import spacy

def chunk_documents(documents, chunk_size=8, overlap=2):
    """
    Segmentiert eine Liste von Texten in Sätze und erstellt Chunks mit Overlap.
    
    :param documents: Liste von Texten (Strings), die segmentiert werden sollen
    :param chunk_size: Anzahl der Sätze pro Chunk (z.B. 8 Sätze pro Chunk)
    :param overlap: Anzahl der Sätze, die zwischen aufeinanderfolgenden Chunks überlappen sollen (z.B. 2 Sätze)
    :return: Liste von Listen, wobei jede innere Liste die Chunks eines Dokuments enthält
    """
    # Lade das deutsche Modell
    nlp = spacy.load('de_core_news_sm')
    
    # Liste zum Speichern der segmentierten und gechunkten Dokumente
    chunked_documents = []
    
    # Verarbeitung und Segmentierung jedes Dokuments in Sätze
    for text in documents:
        doc = nlp(text)
        sentences = list(doc.sents)
        
        # Chunks erstellen
        chunks = []
        for i in range(0, len(sentences), chunk_size - overlap):
            chunk = sentences[i:i + chunk_size]
            chunks.append(chunk)
        
        chunked_documents.append(chunks)
    
    return chunked_documents

def save_chunks_to_file(chunks, filename="chunked_output.txt"):
    """
    Speichert die Chunks in eine Textdatei.
    
    :param chunks: Liste von gechunkten Dokumenten
    :param filename: Der Name der Datei, in die die Chunks geschrieben werden sollen
    """
    with open(filename, 'w', encoding='utf-8') as f:
        for i, doc in enumerate(chunks):
            f.write(f"Dokument {i+1}:\n")
            for j, chunk in enumerate(doc):
                f.write(f"  Chunk {j+1}:\n")
                for sentence in chunk:
                    f.write(str(sentence) + "\n")
                f.write("\n")
            f.write("\n")

# Aufruf der Funktionen
chunked_documents = chunk_documents(cleaned_documents_2, chunk_size=8, overlap=2)
save_chunks_to_file(chunked_documents, filename="chunked_output.txt")

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

def convert_strings_to_embeddings(documents_as_strings):
    """
    Konvertiert eine Liste von Strings in Embeddings.
    
    :param documents_as_strings: Liste von Strings, die in Embeddings umgewandelt werden sollen
    :return: Liste von Embeddings
    """
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
    document_embeddings = embeddings.embed_documents(documents_as_strings)
    return document_embeddings

def convert_chunks_to_strings(chunked_documents):
    """
    Konvertiert eine Liste von gechunkten Spacy-Spans in eine Liste von Strings.
    
    :param chunked_documents: Liste von Listen von Spacy-Spans
    :return: Liste von Strings, wobei jeder String einen Chunk repräsentiert
    """
    documents_as_strings = []
    for doc_chunks in chunked_documents:
        for chunk in doc_chunks:
            chunk_text = " ".join([sentence.text for sentence in chunk])
            documents_as_strings.append(chunk_text)
    return documents_as_strings


# Konvertiere die Chunks in Strings
documents_as_strings = convert_chunks_to_strings(chunked_documents)
print(documents_as_strings)

# Konvertiere die Strings in Embeddings
#document_embeddings = convert_strings_to_embeddings(documents_as_strings)

# Ausgabe der Embeddings für jeden Chunk
#for i, embedding in enumerate(document_embeddings):
#    print(f"Embedding für Chunk {i+1}: {embedding}")

In [ ]:
import os
import pickle

def save_processed_documents(processed_documents, filename):
    # Sicherstellen, dass der Ordner "data/pickle" existiert
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    
    # Speichern der Datei im angegebenen Pfad
    with open(filename, "wb") as file:
        pickle.dump(processed_documents, file)
    
    print(f"Die Datei '{filename}' wurde erfolgreich gespeichert.")

# Beispielnutzung:
#save_processed_documents(document_embeddings, filename="data/pickle/embeddings_for_text.pkl")
save_processed_documents(chunked_documents, filename="data/pickle/list_of_text.pkl")

In [ ]:
def combine_texts_and_embeddings(texts, embeddings):
    """
    Kombiniert eine Liste von Texten mit ihren entsprechenden Embeddings.
    
    :param texts: Liste von Texten (strings)
    :param embeddings: Liste von Embeddings (z.B. Listen von floats)
    
    :return: Eine Liste von Dictionaries, wobei jedes Dictionary die Struktur hat:
             {
                 "id": <Text Index>,
                 "text": <Text>,
                 "embedding": <Embedding>
             }
    """
    combined = []
    for idx, (text, embedding) in enumerate(zip(texts, embeddings)):
        combined.append({
            "id": f"id_{idx}",
            "text": text,
            "embedding": embedding
        })
    
    return combined

combined_list = combine_texts_and_embeddings(documents_as_strings, document_embeddings)
print(combined_list)
save_processed_documents(combined_list, filename="data/pickle/text_embeddings_combine.pkl")

In [ ]:
def strings_to_documents(documents_as_strings):
    # Umwandlung der Strings in Dokumente (in diesem Fall belassen wir die Strings einfach als sie sind)
    documents = [doc for doc in documents_as_strings]
    
    # Ausgabe der Dokumente
    for i, doc in enumerate(documents):
        print(f"Dokument {i+1}:")
        print(doc)
        print("\n" + "-"*50 + "\n")
        
import numpy as np

def convert_and_print_embeddings(document_embeddings):
    """
    Konvertiert eine Liste von Dokumenten-Einbettungen in ein NumPy-Array und gibt die Einbettungen aus.
    
    Args:
    document_embeddings (list): Eine Liste von Dokumenten-Einbettungen.
    
    Returns:
    numpy.ndarray: Das konvertierte NumPy-Array.
    """
    # Konvertiere die Liste in ein NumPy-Array
    embeddings_np = np.array(document_embeddings)
    
    # Ausgabe der Einbettungen
    for i, embedding in enumerate(embeddings_np):
        print(f"Dokument {i+1} Einbettung:")
        print(embedding)
        print("\n" + "-"*50 + "\n")
    
    return embeddings_np
docs_finished = documents_as_strings
emb_finished = convert_and_print_embeddings(document_embeddings)

save_processed_documents(emb_finished, filename="data/pickle/emb_finished.pkl")
save_processed_documents(documents_as_strings, filename="data/pickle/docs_finished.pkl")

In [ ]:
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

def delete_chroma_collection(collection_name='document_embeddings_v7', persist_directory='chroma_db'):
    # Öffne die Chroma-Datenbank mit Persistierung
    client = chromadb.PersistentClient(
        path=persist_directory,
        settings=Settings(),
        tenant=DEFAULT_TENANT,
        database=DEFAULT_DATABASE,
    )
    
    # Überprüfe, ob die Collection existiert, und lösche sie, wenn ja
    existing_collections = [col.name for col in client.list_collections()]
    if collection_name in existing_collections:
        client.delete_collection(name=collection_name)
        print(f"Collection '{collection_name}' erfolgreich gelöscht.")
    else:
        print(f"Collection '{collection_name}' nicht gefunden.")
        
delete_chroma_collection(collection_name='document_embeddings_v7')

In [ ]:
# Test 2
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

def create_chroma_database_v2(embeddings, documents, collection_name='document_embeddings_v7', persist_directory='chroma_db'):
    # Erstelle oder öffne eine Chroma-Datenbank mit Persistierung
    client = chromadb.PersistentClient(
        path=persist_directory,
        settings=Settings(),
        tenant=DEFAULT_TENANT,
        database=DEFAULT_DATABASE,
    )
    
    # Erstelle eine neue Collection in der Datenbank oder lade eine bestehende
    collection = client.get_or_create_collection(name=collection_name)
    
    # Konvertiere Embeddings von numpy array in Liste von Listen
    embeddings_as_list = [embedding.tolist() for embedding in embeddings]
    
    # Dokumente und Embeddings zur Collection hinzufügen
    for i, (embedding, document) in enumerate(zip(embeddings_as_list, documents)):
        collection.add(
            embeddings=[embedding],
            documents=[document],
            ids=[f"doc_{i}"]
        )
    
    print(f"Chroma-Datenbank '{collection_name}' erfolgreich erstellt und {len(embeddings)} Embeddings und Dokumente hinzugefügt.")

# Beispielnutzung:
create_chroma_database_v2(emb_finished, docs_finished)

In [ ]:
# Test 1
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

def create_chroma_database_v2(combined_list, collection_name='document_embeddings_v7', persist_directory='chroma_db'):
    # Erstelle oder öffne eine Chroma-Datenbank mit Persistierung
    client = chromadb.PersistentClient(
        path=persist_directory,
        settings=Settings(),
        tenant=DEFAULT_TENANT,
        database=DEFAULT_DATABASE,
    )
    
    # Erstelle eine neue Collection in der Datenbank oder lade eine bestehende
    collection = client.get_or_create_collection(name=collection_name)
    
    # Füge die Embeddings und Texte (als Dokumente) zur Collection hinzu
    for entry in combined_list:
        embedding_as_list = [entry['embedding']]  # Einzeln in eine Liste einbetten
        collection.add(
            embeddings=embedding_as_list,
            documents=[entry['text']],  # Text als Dokument hinzugefügt
            ids=[entry['id']],
        )
    
    print(f"Chroma-Datenbank '{collection_name}' erfolgreich erstellt und {len(combined_list)} Embeddings hinzugefügt.")

# Beispielnutzung:
create_chroma_database_v2(combined_list)

In [ ]:
import chromadb

def query_first_element(collection_name='document_embeddings_v7', persist_directory='chroma_db', document_id='id_0'):
    # Erstelle oder öffne eine Chroma-Datenbank mit Persistierung
    client = chromadb.PersistentClient(
        path=persist_directory,
        settings=chromadb.config.Settings(),
    )
    
    # Lade die Collection aus der Datenbank
    collection = client.get_collection(name=collection_name)
    
    # Führe die Abfrage durch, um das Dokument und das Embedding mit der spezifischen ID abzurufen
    result = collection.get(ids=[document_id])
    
    # Das Ergebnis anzeigen
    if result and 'documents' in result and result['documents']:
        print(f"Document ID: {document_id}")
        print(f"Text: {result['documents'][0]}")
        if 'embeddings' in result and result['embeddings']:
            print(f"Embedding: {result['embeddings'][0]}")
        else:
            print("No embedding found for this document.")
    else:
        print(f"No document found with ID: {document_id}")

# Beispielnutzung:
query_first_element()

In [ ]:
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer

def query_similar_documents(query_text, collection_name='document_embeddings_v7', persist_directory='chroma_db', model_name='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'):
    # Lade das Embedding-Modell
    model = SentenceTransformer(model_name)
    
    # Erstelle oder öffne eine Chroma-Datenbank mit Persistierung
    client = chromadb.PersistentClient(
        path=persist_directory,
        settings=Settings(),
    )
    
    # Lade die Collection aus der Datenbank
    collection = client.get_collection(name=collection_name)
    
    # Erstelle ein Embedding für den Abfrage-Text
    query_embedding = model.encode(query_text).tolist()
    
    # Führe die Abfrage durch, um ähnliche Dokumente zu finden
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=3,  # Anzahl der zurückgegebenen Ergebnisse, du kannst diese Zahl anpassen
        include=["distances"]  # Fügt den Ähnlichkeits-Score in die Ergebnisse ein
    )
    
    # Debugging: Ausgabe der Struktur von 'results'
    print("Abfrageergebnisse:", results)
    
    # Das Ergebnis anzeigen
    if results and 'documents' in results and results['documents']:
        for i, document in enumerate(results['documents'][0]):
            print(f"Ergebnis {i+1}:")
            print(f"Text: {document}")
            print(f"ID: {results['ids'][0][i]}")
            print(f"Score (Distanz): {results['distances'][0][i]}")
            print()
    else:
        print("Keine ähnlichen Dokumente gefunden.")

# Beispielnutzung:
query_similar_documents("was kommt in den Altglascontainer")